#import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install CatBoost
!pip install kmodes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 3.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,f1_score,roc_auc_score,roc_curve
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,ExtraTreesClassifier,GradientBoostingClassifier,BaggingClassifier,HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn import naive_bayes
from sklearn.ensemble import IsolationForest
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from kmodes.kmodes import KModes
import statsmodels.api as sm
from sklearn.ensemble import StackingRegressor
from sklearn.calibration import CalibratedClassifierCV

#Using Random Under Sampling data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DHS/Diabetes/Data/India/Data Balance/random_under.csv")

In [ ]:
selected_columns = ['hv005', 'hv021', 'hv023', 'hv105', 'hv115', 'hv104', 'hv106', 'hml18',
                    'shb70', 'ha53', 'shb13', 'hv206', 'hv116', 'avg_sys', 'avg_dia', 'bmi']

In [ ]:
#split the data into training data testing sets
X = df[selected_columns]
y = df['final_diabetes']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
DHC_models={
    ################################
    'LR' : LogisticRegression(
          C = 6,
          solver = 'newton-cholesky',
          penalty = 'l2',
          class_weight = 'balanced'
    ),
    ################################
    'RF' : RandomForestClassifier(
          n_estimators = 989,
          criterion = 'entropy',
          max_depth = 16,
          min_samples_split = 7,
          max_features = None,
          bootstrap = True,
          random_state = 28,
          ),
    ################################
    'Adaboost' : AdaBoostClassifier(
        estimator = RandomForestClassifier(
          n_estimators = 989,
          criterion = 'entropy',
          max_depth = 16,
          min_samples_split = 7,
          max_features = None,
          bootstrap = True,
          random_state = 28,
          ),
        n_estimators = 1,
        learning_rate = 0.14,
        algorithm ='SAMME.R',
        random_state =4
    ),

    ################################
    'DT' : DecisionTreeClassifier(
          criterion = 'gini',
          splitter = 'best',
          max_depth =10,
          min_samples_split = 9,
          min_samples_leaf = 1),
    #################################
    'XGB' : XGBClassifier(
          booster = 'gbtree',
          n_estimators = 12,
          max_depth = 6,
          gamma =  1),
    #################################
    'HistGB': HistGradientBoostingClassifier(
             loss = 'log_loss',
            learning_rate = 0.01,
            max_iter = 186,
            max_leaf_nodes = 50 ,
            tol = 0.01,
            random_state = 37,
            class_weight = None
    ),
    #################################
    'GBC' : GradientBoostingClassifier(
          loss ='exponential',
          learning_rate = 0.46,
          n_estimators =89,
          min_samples_split= 3,
          max_depth = 3,
          random_state = 9,
    ),
    #################################
    'Bagging' : BaggingClassifier(
              n_estimators =  149,
              bootstrap = True,
              bootstrap_features = False,
              oob_score = False,
              n_jobs = 1,
              random_state = 27,
    ),
    #################################
    'ExtraT':ExtraTreesClassifier(
            n_estimators = 46,
            min_samples_split = 4,
            max_leaf_nodes = 17,
            min_impurity_decrease= 0.09,
            n_jobs = 8,
            random_state = 20
    ),

}

In [ ]:
Final_Results=pd.DataFrame(columns=['Models','Accuracy','Precision','Recall','F1-Score','Roc_Auc','Specificity','Sensitivity','Confusion Metrix'])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score, roc_auc_score, confusion_matrix

'''This model is to evaluate the models and use different evaulating method to evaluate the models'''

def evaluate_model(model, X, y, n_splits=10):
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=42)
    accuracies, precisions, recalls,f1s,cm,specificitys, sensitivitys,roc_aucs= [], [], [],[],[],[],[],[]

    for train_idx, test_idx in kf.split(X):
      # take the location of the splited data and access it by the index
      X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
      X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
      # train the model
      model.fit(X_train, y_train)
      # make prediction
      y_pred = model.predict(X_test)

      # evaluation scores
      accuracy = accuracy_score(y_test, y_pred)
      precision = precision_score(y_test, y_pred,zero_division=1)
      recall = recall_score(y_test, y_pred)
      f1= f1_score(y_test, y_pred)
      cms = confusion_matrix(y_test, y_pred)  # Calculate confusion matrix
      roc_auc=roc_auc_score(y_test, y_pred)

      # calculate the specificity and sensitivity
      tn, fp, fn, tp = cms.ravel()
      specificity = tn / (tn + fp)
      sensitivity = tp / (tp + fn)



      # append all the results
      accuracies.append(accuracy)
      precisions.append(precision)
      recalls.append(recall)
      f1s.append(f1)
      cm.append(cms)                   # Add confusion matrix to list
      specificitys.append(specificity) #add specificity score
      sensitivitys.append(sensitivity) #add sensitivity scores to the list
      roc_aucs.append(roc_auc) #add roc_auc score to the list


    return (format(sum(accuracies)/n_splits, '.3f'), format(sum(precisions)/n_splits, '.3f'),
            format(sum(recalls)/n_splits, '.3f'),format(sum(f1s)/n_splits, '.3f'),format(sum(roc_aucs)/n_splits, '.3f'), sum(cm)/n_splits ,format(sum(specificitys)/n_splits, '.3f'),
            format(sum(sensitivitys)/n_splits, '.3f'))


In [ ]:
def modelApplied(Model,X_train , y_train, n_splits=10,MN='model_name',):
  #put the X , Y value here

  accuracy, precision, recall,f1,roc_auc,cm,specificity,sensitivity = evaluate_model(Model,X_train , y_train, n_splits=10)

  new_row=[MN,accuracy,precision,recall,f1,roc_auc,specificity, sensitivity,cm]

  Final_Results.loc[len(Final_Results)]=new_row

  print('DONE!!')

In [ ]:
for k,v in DHC_models.items():
    modelApplied(v,X_train, y_train,MN= k )

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #2. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=1.00631e-16): result may not be accurate.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_glm/_newton_solver.py:195: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.10/dist-p

DONE!!
DONE!!
DONE!!
DONE!!
DONE!!
DONE!!
DONE!!
DONE!!
DONE!!


In [ ]:
Final_Results

,Models,Accuracy,Precision,Recall,F1-Score,Roc_Auc,Specificity,Sensitivity,Confusion Metrix
0,LR,0.948,0.967,0.927,0.947,0.948,0.968,0.927,"[[752.4, 24.8], [56.5, 721.5]]"
1,RF,0.958,0.976,0.939,0.957,0.958,0.977,0.939,"[[759.1, 18.1], [47.2, 730.8]]"
2,Adaboost,0.959,0.976,0.940,0.958,0.959,0.977,0.940,"[[759.6, 17.6], [46.9, 731.1]]"
3,DT,0.954,0.973,0.935,0.953,0.954,0.974,0.935,"[[757.0, 20.2], [50.7, 727.3]]"
4,XGB,0.958,0.977,0.937,0.957,0.958,0.978,0.937,"[[760.2, 17.0], [48.9, 729.1]]"
5,HistGB,0.957,0.969,0.943,0.956,0.957,0.970,0.943,"[[753.9, 23.3], [44.0, 734.0]]"
6,GBC,0.957,0.968,0.945,0.956,0.957,0.968,0.945,"[[752.5, 24.7], [42.6, 735.4]]"
7,Bagging,0.956,0.967,0.943,0.955,0.956,0.968,0.943,"[[752.5, 24.7], [44.3, 733.7]]"
8,ExtraT,0.953,0.997,0.909,0.951,0.953,0.998,0.909,"[[775.3, 1.9], [71.0, 707.0]]"


In [ ]:
Final_Results.to_csv("/content/drive/MyDrive/DHS/Diabetes/Data/India/Model Implementation/optimize_All_Model_implement.csv")

In [ ]:
Final_Results

,Models,Accuracy,Precision,Recall,F1-Score,Roc_Auc,Specificity,Sensitivity,Confusion Metrix
0,LR,0.948,0.967,0.927,0.947,0.948,0.968,0.927,"[[752.4, 24.8], [56.5, 721.5]]"
1,RF,0.958,0.976,0.939,0.957,0.958,0.977,0.939,"[[759.1, 18.1], [47.2, 730.8]]"
2,Adaboost,0.959,0.976,0.940,0.958,0.959,0.977,0.940,"[[759.6, 17.6], [46.9, 731.1]]"
3,DT,0.954,0.973,0.935,0.953,0.954,0.974,0.935,"[[757.0, 20.2], [50.7, 727.3]]"
4,XGB,0.958,0.977,0.937,0.957,0.958,0.978,0.937,"[[760.2, 17.0], [48.9, 729.1]]"
5,HistGB,0.957,0.969,0.943,0.956,0.957,0.970,0.943,"[[753.9, 23.3], [44.0, 734.0]]"
6,GBC,0.957,0.968,0.945,0.956,0.957,0.968,0.945,"[[752.5, 24.7], [42.6, 735.4]]"
7,Bagging,0.956,0.967,0.943,0.955,0.956,0.968,0.943,"[[752.5, 24.7], [44.3, 733.7]]"
8,ExtraT,0.953,0.997,0.909,0.951,0.953,0.998,0.909,"[[775.3, 1.9], [71.0, 707.0]]"
